In [ ]:
import os
import json
from pathlib import Path

# Path to cropped bodies and JSON file
CROPPED_BODY_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented\cropped_bodies_new"
BODY_JSON_PATH = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented\body_bboxes.json"

# Load existing JSON
with open(BODY_JSON_PATH, 'r') as f:
    detections = json.load(f)

# Dictionary to store updated detections
updated_detections = {}

# Process files in cropped body directory
for root, _, files in os.walk(CROPPED_BODY_DIR):
    for file in files:
        if "_devil_0" in file and file.endswith('.jpg'):
            old_path = os.path.join(root, file)
            new_file = file.replace("_devil_0", "")
            new_path = os.path.join(root, new_file)

            # If the new filename already exists, remove it before renaming
            if os.path.exists(new_path):
                os.remove(new_path)  # Delete existing file to avoid conflict
                print(f"🗑️ Deleted existing file: {new_path}")

            # Rename the file
            os.rename(old_path, new_path)
            print(f"✅ Renamed: {old_path} → {new_path}")

            # Update JSON entries
            if old_path in detections:
                updated_detections[new_path] = detections.pop(old_path)

# Save updated JSON
with open(BODY_JSON_PATH, 'w') as f:
    json.dump(updated_detections, f, indent=4)

print("\n✅ Filename cleanup and JSON update completed!")


In [ ]:
import os
import json
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO

# ------------------------
# CONFIGURATION
# ------------------------
CROPPED_BODY_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1\cropped_bodies"
MASKS_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented\split_original"
OUTPUT_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1"
BODY_JSON_PATH = os.path.join(OUTPUT_DIR, "body_face_bboxes.json")
UPDATED_JSON_PATH = os.path.join(OUTPUT_DIR, "updated_body_face_bboxes.json")  # New JSON for body + face info

# YOLO Face Model
YOLO_FACE_MODEL_PATH = r"C:\Users\Jaylen LI\yolov8-face\train\weights\best.pt"

# Output directories for cropped & masked faces
CROPPED_FACE_DIR = os.path.join(OUTPUT_DIR, "cropped_faces")
MASKED_FACE_DIR = os.path.join(OUTPUT_DIR, "masked_faces")

for folder in [CROPPED_FACE_DIR, MASKED_FACE_DIR]:
    os.makedirs(folder, exist_ok=True)

# Load YOLO Face Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
face_model = YOLO(YOLO_FACE_MODEL_PATH).to(device)

# Load existing JSON with body bounding boxes
if not os.path.exists(BODY_JSON_PATH):
    print(f"❌ ERROR: Missing body JSON: {BODY_JSON_PATH}")
    exit()

with open(BODY_JSON_PATH, 'r') as f:
    body_detections = json.load(f)

# New dictionary to store body + face coordinates relative to original image
updated_detections = {}

# ------------------------
# FUNCTION: Load Mask
# ------------------------
def load_mask(mask_base_dir, image_path):
    """Loads a VOS mask and ensures correct path formatting."""
    subfolder = os.path.basename(os.path.dirname(image_path))  # Extracts subfolder name
    mask_path = os.path.join(mask_base_dir, subfolder, Path(image_path).stem + ".png")

    if not os.path.exists(mask_path):
        print(f"⚠️ Warning: Mask file not found: {mask_path}")
        return None

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Warning: Failed to load mask: {mask_path}")
        return None
    return (mask > 0).astype(np.uint8)  # Convert to binary mask

# ------------------------
# FUNCTION: Detect & Crop Faces with Corrected Coordinates
# ------------------------
def detect_and_crop_faces():
    """Detects faces from already cropped body images and maps them back to original coordinates."""
    
    print("\n🟢 Starting Face Cropping & JSON Update...")

    for cropped_body_path, metadata in tqdm(body_detections.items()):
        image = cv2.imread(cropped_body_path)
        original_image_path = metadata["original_image"]
        body_bbox = metadata["bbox"]  # Bounding box of body in original image

        if image is None:
            print(f"❌ Error loading {cropped_body_path}")
            continue

        # YOLO Face Detection
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_model.predict(source=img_rgb, imgsz=640)
        predictions = results[0].boxes.data.cpu().numpy()

        if len(predictions) == 0:
            print(f"⚠️ No face detected for {cropped_body_path}")
            continue  

        for pred in predictions:
            x1_face, y1_face, x2_face, y2_face, conf, cls = pred[:6]

            print(f"🔍 Face detected in {cropped_body_path} with confidence {conf:.2f}")

            if conf < 0.5:
                print(f"⚠️ Confidence too low ({conf:.2f}), skipping {cropped_body_path}")
                continue

            # Convert face bbox from body image space → original full image space
            x1_face_global = int(body_bbox[0] + x1_face)
            y1_face_global = int(body_bbox[1] + y1_face)
            x2_face_global = int(body_bbox[0] + x2_face)
            y2_face_global = int(body_bbox[1] + y2_face)

            # Preserve dataset structure in output
            relative_body_path = os.path.relpath(cropped_body_path, CROPPED_BODY_DIR)
            output_face_dir = os.path.join(CROPPED_FACE_DIR, os.path.dirname(relative_body_path))
            os.makedirs(output_face_dir, exist_ok=True)

            output_face_file = os.path.join(output_face_dir, Path(cropped_body_path).stem + "_face.jpg")
            cv2.imwrite(output_face_file, image[int(y1_face):int(y2_face), int(x1_face):int(x2_face)])

            print(f"✅ Saved cropped face: {output_face_file}")  

            # Load the correct mask from original image path
            mask = load_mask(MASKS_DIR, original_image_path)

            if mask is not None:
                mask_face = np.zeros_like(image[int(y1_face):int(y2_face), int(x1_face):int(x2_face)])
                mask_roi = mask[y1_face_global:y2_face_global, x1_face_global:x2_face_global]
                mask_face[mask_roi > 0] = image[y1_face:y2_face, x1_face:x2_face][mask_roi > 0]

                output_masked_face_dir = os.path.join(MASKED_FACE_DIR, os.path.dirname(relative_body_path))
                os.makedirs(output_masked_face_dir, exist_ok=True)

                masked_output_face = os.path.join(output_masked_face_dir, Path(cropped_body_path).stem + "_face.jpg")
                cv2.imwrite(masked_output_face, mask_face)

                print(f"✅ Saved masked face: {masked_output_face}")

            # Update JSON with face coordinates mapped back to original image
            updated_detections[output_face_file] = {
                "original_image": original_image_path,
                "bbox_body": body_bbox,
                "bbox_face": [x1_face_global, y1_face_global, x2_face_global, y2_face_global]
            }
            print(f"✅ Adding to JSON: {output_face_file} -> {x1_face_global, y1_face_global, x2_face_global, y2_face_global}")

    updated_json_path = os.path.join(OUTPUT_DIR, "updated_body_face_bboxes.json")
    with open(updated_json_path, 'w') as f:
        json.dump(updated_detections, f, indent=4)

    print("\n✅ Face Cropping & JSON Update Completed!")

# ------------------------
# EXECUTION
# ------------------------
detect_and_crop_faces()

print("\n✅ Face detection completed with JSON update for original coordinates!")
#working

In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

# ------------------------
# CONFIGURATION
# ------------------------
BASE_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented\split_original"
MASKS_DIR = BASE_DIR  # Ensure masks are in the same structure

CROPPED_FACE_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1\cropped_faces"
MASKED_FACE_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces"
os.makedirs(MASKED_FACE_DIR, exist_ok=True)

# ✅ FIXED JSON PATH
JSON_PATH = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1\updated_body_face_bboxes.json"

# ------------------------
# FUNCTION: Load Mask
# ------------------------
def load_mask(mask_path):
    """Loads a VOS mask as a binary numpy array."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        return None
    return (mask > 0).astype(np.uint8)  # Convert to binary mask

# ------------------------
# FUNCTION: Apply Mask to Cropped Face
# ------------------------
def apply_mask_to_faces():
    """Applies the corresponding VOS mask to each cropped face."""
    
    # ✅ Check if JSON exists
    if not os.path.exists(JSON_PATH):
        print(f"❌ ERROR: JSON file not found: {JSON_PATH}")
        return

    # Load bounding box data
    with open(JSON_PATH, 'r') as f:
        detections = json.load(f)

    if not detections:
        print(f"❌ ERROR: JSON file is empty: {JSON_PATH}")
        return

    for face_image_path, metadata in tqdm(detections.items()):
        original_image_path = metadata["original_image"]
        bbox_face = metadata["bbox_face"]

        # Load face image
        face_image = cv2.imread(face_image_path)
        if face_image is None:
            print(f"❌ Error loading {face_image_path}")
            continue
        
        # Load corresponding mask
        mask_path = original_image_path.replace("images", "masks").replace(".jpg", ".png")
        mask = load_mask(mask_path)
        if mask is None:
            print(f"⚠️ No mask found for {original_image_path}, skipping mask application.")
            continue

        # Extract face bounding box coordinates
        x1, y1, x2, y2 = bbox_face
        mask_face = mask[y1:y2, x1:x2]

        if mask_face.shape[:2] != face_image.shape[:2]:
            print(f"⚠️ Mask size mismatch for {face_image_path}, skipping.")
            continue

        # Apply Mask
        masked_face = np.zeros_like(face_image)
        masked_face[mask_face > 0] = face_image[mask_face > 0]

        # Save Masked Face
        output_file = face_image_path.replace(CROPPED_FACE_DIR, MASKED_FACE_DIR)
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        cv2.imwrite(output_file, masked_face)

        print(f"✅ Mask applied: {output_file}")

# ------------------------
# EXECUTION
# ------------------------
apply_mask_to_faces()
print("\n✅ Masking process for cropped faces completed!")
#working

In [ ]:
import os
import json
import cv2
import torch
import numpy as np
from tqdm import tqdm
from pathlib import Path
from ultralytics import YOLO  # Ensure Ultralytics package is installed

# ------------------------
# CONFIGURATION
# ------------------------
OUTPUT_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1"
FINAL_JSON_PATH = os.path.join(OUTPUT_DIR, "test_body_face_bboxes.json")  # JSON for test set bounding boxes

# YOLO Face Model
YOLO_FACE_MODEL_PATH = r"C:\Users\Jaylen LI\yolov8-face\train\weights\best.pt"

# Ensure directories exist
CROPPED_BODY_DIR = os.path.join(OUTPUT_DIR, "cropped_bodies", "test")  # Input for face cropping
CROPPED_FACE_DIR = os.path.join(OUTPUT_DIR, "cropped_faces", "test")   # Output for cropped faces
MASKED_FACE_DIR = os.path.join(OUTPUT_DIR, "masked_faces", "test")     # Output for masked faces

for folder in [CROPPED_FACE_DIR, MASKED_FACE_DIR]:
    os.makedirs(folder, exist_ok=True)

# Load YOLO Face Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
face_model = YOLO(YOLO_FACE_MODEL_PATH).to(device)

# ------------------------
# FUNCTION: Detect & Crop Faces (TEST SET)
# ------------------------
def detect_and_crop_test_faces():
    """Detects faces using YOLOv8 on already cropped full-body test images."""
    
    print("\n🔍 Processing Test Set for Face Detection...")
    
    # Load existing JSON with body bounding boxes
    if not os.path.exists(FINAL_JSON_PATH):
        print(f"❌ ERROR: Missing test JSON file: {FINAL_JSON_PATH}. Run body detection first!")
        return

    with open(FINAL_JSON_PATH, 'r') as f:
        body_detections = json.load(f)

    face_detections = {}
    
    for cropped_body_path, metadata in tqdm(body_detections.items()):
        image = cv2.imread(cropped_body_path)
        original_image_path = metadata["original_image"]
        body_bbox = metadata["bbox"]  # Bounding box of body in original image

        if image is None:
            print(f"❌ Error loading {cropped_body_path}")
            continue

        # YOLO Face Detection
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_model.predict(source=img_rgb, imgsz=640)
        predictions = results[0].boxes.data.cpu().numpy()

        if len(predictions) == 0:
            print(f"⚠️ No face detected in {cropped_body_path}")
            continue  

        object_count = 0
        for pred in predictions:
            x1_face, y1_face, x2_face, y2_face, conf, cls = pred[:6]

            print(f"🔍 Face detected in {cropped_body_path} with confidence {conf:.2f}")

            if conf < 0.3:  # Lower threshold for test set
                print(f"⚠️ Confidence too low ({conf:.2f}), skipping {cropped_body_path}")
                continue

            cropped_face = image[int(y1_face):int(y2_face), int(x1_face):int(x2_face)]
            if cropped_face.size == 0:
                print(f"❌ Invalid face crop for {cropped_body_path}, skipping.")
                continue

            # Preserve dataset structure in output
            relative_body_path = os.path.relpath(cropped_body_path, CROPPED_BODY_DIR)
            output_face_dir = os.path.join(CROPPED_FACE_DIR, os.path.dirname(relative_body_path))
            os.makedirs(output_face_dir, exist_ok=True)

            output_face_file = os.path.join(output_face_dir, Path(cropped_body_path).stem + "_face.jpg")
            cv2.imwrite(output_face_file, cropped_face)

            print(f"✅ Saved cropped face: {output_face_file}")  

            # Convert face bbox from body image space → original full image space
            x1_face_global = int(body_bbox[0] + x1_face)
            y1_face_global = int(body_bbox[1] + y1_face)
            x2_face_global = int(body_bbox[0] + x2_face)
            y2_face_global = int(body_bbox[1] + y2_face)

            # Update JSON with face coordinates mapped back to original image
            face_detections[output_face_file] = {
                "original_image": original_image_path,
                "bbox_body": body_bbox,
                "bbox_face": [x1_face_global, y1_face_global, x2_face_global, y2_face_global]
            }

    # Save updated JSON with face coordinates
    updated_json_path = os.path.join(OUTPUT_DIR, "test_face_bboxes.json")
    with open(updated_json_path, 'w') as f:
        json.dump(face_detections, f, indent=4)

    print(f"\n✅ Face Cropping Completed! JSON saved: {updated_json_path}")

# ------------------------
# EXECUTION
# ------------------------
detect_and_crop_test_faces()

print("\n✅ Test Set Face Detection & Cropping Completed!")
#working

In [13]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm
from pathlib import Path

# ------------------------
# CONFIGURATION
# ------------------------
OUTPUT_DIR = r"D:\Master's Research\yolo+vos+arcface\dataset_augmented1"
MASKS_DIR = os.path.join(OUTPUT_DIR, "test", "masks")  # Path to test set masks
BODY_JSON_PATH = os.path.join(OUTPUT_DIR, "test_body_face_bboxes.json")  # Body bboxes
FACE_JSON_PATH = os.path.join(OUTPUT_DIR, "test_face_bboxes.json")  # Face bboxes

# Input directories (original cropped images)
CROPPED_BODY_DIR = os.path.join(OUTPUT_DIR, "cropped_bodies", "test")
CROPPED_FACE_DIR = os.path.join(OUTPUT_DIR, "cropped_faces", "test")

# Output directories (masked images)
MASKED_BODY_DIR = os.path.join(OUTPUT_DIR, "masked_bodies", "test")
MASKED_FACE_DIR = os.path.join(OUTPUT_DIR, "masked_faces", "test")

for folder in [MASKED_BODY_DIR, MASKED_FACE_DIR]:
    os.makedirs(folder, exist_ok=True)

# ------------------------
# FUNCTION: Load Mask
# ------------------------
def load_mask(mask_path):
    """Loads a VOS mask as a binary numpy array."""
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        print(f"⚠️ Warning: Mask file missing {mask_path}")
        return None
    return (mask > 0).astype(np.uint8)  # Convert to binary mask

# ------------------------
# FUNCTION: Apply Mask to Image
# ------------------------
def apply_mask(image, mask):
    """Applies a mask to an image, keeping only the masked regions."""
    if image.shape[:2] != mask.shape[:2]:  # Resize mask to match image size
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)
    masked_image = np.zeros_like(image)
    masked_image[mask > 0] = image[mask > 0]
    return masked_image

# ------------------------
# FUNCTION: Apply Mask to Cropped Bodies (Fixed)
# ------------------------
def apply_mask_to_bodies():
    """Applies VOS masks to test cropped bodies."""
    if not os.path.exists(BODY_JSON_PATH):
        print(f"❌ ERROR: Missing test body JSON: {BODY_JSON_PATH}")
        return

    with open(BODY_JSON_PATH, 'r') as f:
        body_detections = json.load(f)

    for cropped_body_path, metadata in tqdm(body_detections.items()):
        original_image_path = metadata["original_image"]
        bbox_body = metadata["bbox"]

        # Load cropped body image
        body_image = cv2.imread(cropped_body_path)
        if body_image is None:
            print(f"❌ Error loading {cropped_body_path}")
            continue

        # Find corresponding mask (must match the original full image size)
        mask_path = original_image_path.replace("images", "masks").replace(".jpg", ".png")
        full_mask = load_mask(mask_path)
        if full_mask is None:
            continue

        # Extract the body mask region from the full mask
        x1, y1, x2, y2 = bbox_body
        body_mask = full_mask[y1:y2, x1:x2]

        if body_mask.shape[:2] != body_image.shape[:2]:  # Ensure mask matches cropped body
            body_mask = cv2.resize(body_mask, (body_image.shape[1], body_image.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Apply mask to the cropped body
        masked_body = apply_mask(body_image, body_mask)
        if masked_body is None:
            continue

        # Save masked body
        relative_body_path = os.path.relpath(cropped_body_path, CROPPED_BODY_DIR)
        output_masked_body = os.path.join(MASKED_BODY_DIR, relative_body_path)
        os.makedirs(os.path.dirname(output_masked_body), exist_ok=True)
        cv2.imwrite(output_masked_body, masked_body)

        print(f"✅ Masked body saved: {output_masked_body}")

# ------------------------
# FUNCTION: Apply Mask to Cropped Faces (Working)
# ------------------------
def apply_mask_to_faces():
    """Applies VOS masks to test cropped faces."""
    if not os.path.exists(FACE_JSON_PATH):
        print(f"❌ ERROR: Missing test face JSON: {FACE_JSON_PATH}")
        return

    with open(FACE_JSON_PATH, 'r') as f:
        face_detections = json.load(f)

    for cropped_face_path, metadata in tqdm(face_detections.items()):
        original_image_path = metadata["original_image"]
        bbox_face = metadata["bbox_face"]

        # Load cropped face image
        face_image = cv2.imread(cropped_face_path)
        if face_image is None:
            print(f"❌ Error loading {cropped_face_path}")
            continue

        # Find corresponding mask
        mask_path = original_image_path.replace("images", "masks").replace(".jpg", ".png")
        full_mask = load_mask(mask_path)
        if full_mask is None:
            continue

        # Extract face region mask from full mask
        x1, y1, x2, y2 = bbox_face
        mask_face = full_mask[y1:y2, x1:x2]

        if mask_face.shape[:2] != face_image.shape[:2]:  # Resize mask if needed
            mask_face = cv2.resize(mask_face, (face_image.shape[1], face_image.shape[0]), interpolation=cv2.INTER_NEAREST)

        # Apply mask to the cropped face
        masked_face = apply_mask(face_image, mask_face)
        if masked_face is None:
            continue

        # Save masked face
        relative_face_path = os.path.relpath(cropped_face_path, CROPPED_FACE_DIR)
        output_masked_face = os.path.join(MASKED_FACE_DIR, relative_face_path)
        os.makedirs(os.path.dirname(output_masked_face), exist_ok=True)
        cv2.imwrite(output_masked_face, masked_face)

        print(f"✅ Masked face saved: {output_masked_face}")

# ------------------------
# EXECUTION
# ------------------------
print("\n🔍 Applying VOS masks to cropped bodies...")
apply_mask_to_bodies()

print("\n🔍 Applying VOS masks to cropped faces...")
apply_mask_to_faces()

print("\n✅ Masking process for test set completed!")



🔍 Applying VOS masks to cropped bodies...


  5%|████▏                                                                             | 6/116 [00:00<00:01, 57.96it/s]

✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Case\frame_0004_5421_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0000_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0003_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0004_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0010_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0034_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Chev\frame_0035_5417_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Colet\frame_00

 17%|█████████████▉                                                                   | 20/116 [00:00<00:01, 64.38it/s]

✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Colet\frame_0033_5394_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\DarrenHayes\frame_0010_0001_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\frame_0002_5403_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\frame_0002_5458_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\frame_0005_5458_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\frame_0007_5458_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\frame_0008_5458_body.jpg
✅ Masked body saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_bodies\test\Dart\f

...

✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0006_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0008_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0010_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0036_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0037_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Komatsu\frame_0040_5428_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Manchester\frame_0003_5467_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\

 88%|███████████████████████████████████████████████████████████████████████▊          | 70/80 [00:00<00:00, 81.42it/s]

✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0006_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0007_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0016_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0031_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0033_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0034_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Maple Leaf\frame_0039_5411_body_face.jpg
✅ Masked face saved: D:\Master's Research

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:01<00:00, 79.99it/s]

✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Pacific\frame_0000_5469_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Pacific\frame_0017_5469_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Republic\frame_0001_5480_body_face.jpg
✅ Masked face saved: D:\Master's Research\yolo+vos+arcface\dataset_augmented1\masked_faces\test\Sentinel\frame_0008_5474_body_face.jpg

✅ Masking process for test set completed!
